# 简介

**适用于本colab关联的sovits2.0模型，1.0/2.0模型、脚本均不通用**

**[1.0模型合成](https://colab.research.google.com/drive/14GT8uWM9IATjVylYNGGVXiKWHaDHZSTf)**

[一键制作数据集](https://colab.research.google.com/drive/1avWZ_N5BsQcq45XkwQkDpmp912CLZS0n?usp=drive_open#scrollTo=xx2oAf90btEy)

[一键训练](https://colab.research.google.com/drive/1DexYpwWIdD_RRqQ165l-YoWMzFAHIbPy?usp=drive_open)

**支持一键合成长时间的音频（5min以上），建议使用GPU（CPU比较慢）**

**请确保每一步均按顺序完成**

**本colab高度自动化，完全按提示操作即可合成，不要自己新建路径等；报错可能是网络原因，清除数据从头开始即可**

# 配置环境

In [1]:
#@title 克隆仓库、安装包
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install --upgrade pip
!git clone -b colab https://github.com/xzy-git/sovits_f0_infer
%cd sovits_f0_infer
!pip install -r requirements.txt
!mkdir pth
!mkdir raw
!mkdir results
%cd wav_temp
!mkdir input
!mkdir output
%cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 12 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (316 kB/s)
Selecting p

In [2]:
#@title 导入包
!pip install demjson pydub pyworld
import logging
import os
import subprocess

import demjson
import soundfile
import torch

from sovits import infer_tool
from wav_temp import merge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached demjson-2.2.4.tar.gz (131 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.0/212.0 kB 14.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for demjson: filename=demjson-2.2.4-py3-none-any.whl size=73565 sha256=0c6371e3b002f835edc25093a656115db756a847964db8b55e41216dcbd9ed0f
  Stored in directory: /root/.cache/pip/wheels/9f/48/75/cb3fd42f8583b0b0ebc45ad6af1ff9eeb3664fdc6f76589f43
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609451 sha256=51b20fa71d4fb667173835f2c4de4a850077822213f68d904950d6c32abd802d
  Stored in directory: /root/.cache/pip/wheels/54/15/09/232d7d255a8301c2a4a54caed65c7f064dc2bd6705a36b477d
Successfully built demjson pyworld


# 加载模型

## 下载内容编码器

In [3]:
# 这个东西是https://github.com/bshall/hubert/releases/tag/v0.1 的hubert-soft-0d54a1f4.pt，可以自己替换来源、但是不能换其他模型（路径自己改）。
#!gdown --id '1cA37nsiSnsouF2TJkaXb3_VoA-rbifTu' --output /content/sovits_f0_infer/pth/hubert.pt
!wget https://huggingface.co/spaces/xiaolang/sovits_midi_dev/resolve/main/hubert.pt -O /content/sovits_f0_infer/pth/hubert.pt

--2022-10-16 10:21:54--  https://huggingface.co/spaces/xiaolang/sovits_midi_dev/resolve/main/hubert.pt
Resolving huggingface.co (huggingface.co)... 54.173.5.192, 54.210.225.113, 52.5.54.249, ...
Connecting to huggingface.co (huggingface.co)|54.173.5.192|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/28/d8/28d830556bcdd0cb46c4fede42ac3941296568acdd6086982580ca95e028d6ca/e82e7d079df05fe3aa535f6f7d42d309bdae1d2a53324e2b2386c56721f4f649?response-content-disposition=attachment%3B%20filename%3D%22hubert.pt%22&Expires=1666172136&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzI4L2Q4LzI4ZDgzMDU1NmJjZGQwY2I0NmM0ZmVkZTQyYWMzOTQxMjk2NTY4YWNkZDYwODY5ODI1ODBjYTk1ZTAyOGQ2Y2EvZTgyZTdkMDc5ZGYwNWZlM2FhNTM1ZjZmN2Q0MmQzMDliZGFlMWQyYTUzMzI0ZTJiMjM4NmM1NjcyMWY0ZjY0OT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPWF0dGFjaG1lbnQlM0IlMjBmaWxlbmFtZSUzRCUyMmh1YmVydC5wdCUyMiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1Ro

## 加载生成器

如果要**替换自己的模型**，请自行上传**配套的2.0版本的configs、G_xxx.pth**

**使用训练专栏输出到云盘的，改一下名称、无需更改路径**

将**自己的配置json（上一篇生成了的）**上传至/content/sovits_f0_infer/configs/文件夹
将**自己的模型（上一篇生成了的）**上传至/content/sovits_f0_infer/pth文件夹

In [4]:
from google.colab import drive

#@markdown 是否使用谷歌盘内模型（不勾选则自动下载猫雷模型）
g_drive = True #@param {type:"boolean"}
if g_drive:
  drive.mount('/content/drive/')
  #@markdown 若使用，自行更改文件夹名、模型名
  config_path = "/content/drive/MyDrive/nyarumul/config.json" #@param {type:"string"}
  model_path = '/content/drive/MyDrive/nyarumul/G_38000.pth' #@param {type:"string"}
else:
  # 这个东西是https://huggingface.co/spaces/innnky/soft-vits-singingvc 的G.pth（猫雷），可以换成自己的模型（必须是按照sovits方式训练出的其他角色模型）
  !wget https://huggingface.co/spaces/xiaolang/sovits_f0/resolve/main/152_epochs.pth -O /content/sovits_f0_infer/pth/G.pth  
  config_path = "./configs/nyarumul.json"
  model_path = "/content/sovits_f0_infer/pth/G.pth"

# 加载sovits模型、参数
net_g_ms, hubert_soft, feature_input, hps_ms = infer_tool.load_model(model_path, config_path)
speakers = demjson.decode_file(config_path)["speakers"]
target_sample = hps_ms.data.sampling_rate

Mounted at /content/drive/


# 声音转换

支持{1、2}**任选一个方式**的声音转换！
支持**10s以上5分钟以内**的音频（再久合成时间会过长）。
上传到/content/sovits_f0_infer/raw文件夹，支持自动合成歌曲

使用[spleeter](https://github.com/deezer/spleeter)的2stems模式分离歌曲，自动生成这两个文件。（请自行阅读官方使用文档）

spleeter separate -p spleeter:2stems -o output audio_example.mp3

**结果自动输出至results文件夹。**自行下载，无预览
mp3为自动合成的带伴奏歌曲，out_vits为纯人声。

1、使用参考音频

In [5]:
!gdown --id '10JQMPdzp0gjg9cVVersxVZWhIr4UwrFF' --output /content/sovits_f0_infer/raw/vocals.wav

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=10JQMPdzp0gjg9cVVersxVZWhIr4UwrFF
To: /content/sovits_f0_infer/raw/vocals.wav
100% 882k/882k [00:00<00:00, 167MB/s]


2、使用上传音频

自行上传至raw文件夹（单声道，22050hz，wav格式）

In [6]:
#@title 加载Google云端硬盘，挂云盘直接往raw拖文件也可以
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3、合成音频

In [33]:
import torchaudio
#@markdown 角色id——猫雷模型：0号为猫雷，2号为云灏，3号为即霜，4号为奕兰秋

#@markdown 源音频为女声，**目标音色为男声**时，**降3-6key**效果较好，反之同理

#@markdown 角色id
speaker_id = "0" #@param {type:"string"}
#@markdown **可为正负（升降n个半音）**
vc_transform = "12" #@param {type:"string"}
#@markdown 人声文件名（不带.wav）
clean_name = "00429" #@param {type:"string"}

out_audio_name = clean_name
# 可填写音源文件列表，音源文件格式为wav，放置于raw文件夹下
clean_names = [clean_name]
# 合成多少歌曲时，若半音数量不足、自动补齐相同数量（按第一首歌的半音）
trans = [int(vc_transform)]  # 加减半音数（可为正负）s
# 每首歌同时输出的speaker_id
id_list = [int(speaker_id)]



dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
infer_tool.fill_a_to_b(trans, clean_names)  # 自动补齐
input_wav_path = "./wav_temp/input"
out_wav_path = "./wav_temp/output"
infer_tool.mkdir([input_wav_path, out_wav_path])
print("mis连续超过10%时，考虑升降半音\n")
# 遍历列表
for clean_name, tran in zip(clean_names, trans):
    raw_audio_path = f"./raw/{clean_name}.wav"
    infer_tool.format_wav(raw_audio_path, target_sample)
    for spk_id in id_list:
        # 清除缓存文件
        infer_tool.del_temp_wav("./wav_temp")
        var_list = []
        mis_list = []
        out_audio_name = clean_name

        proc = subprocess.Popen(
            f"python ./sovits/slicer.py {raw_audio_path} --out_name {out_audio_name} --out {input_wav_path}  --db_thresh -30",
            shell=True).wait()
        # shutil.copy(raw_audio_path, f"{input_wav_path}/{out_audio_name}-00.wav")
        count = 0
        file_list = os.listdir(input_wav_path)
        len_file_list = len(file_list)
        for file_name in file_list:
            raw_path = f"{input_wav_path}/{file_name}"
            out_path = f"{out_wav_path}/{file_name}"

            a,s=torchaudio.load(raw_path)
            out_audio, out_sr = infer_tool.infer(a,s, spk_id, tran, net_g_ms, hubert_soft, feature_input)
            soundfile.write(out_path, out_audio, target_sample)

            mistake, var = infer_tool.calc_error(raw_path, out_path, tran, feature_input)
            mis_list.append(mistake)
            var_list.append(var)
            count += 1
            print(f"{file_name}: {round(100 * count / len_file_list, 2)}%  mis:{mistake} var:{var}")
        print(
            f"分段误差参考：0.3优秀，0.5左右合理，少量0.8-1可以接受\n若偏差过大，请调整升降半音数；多次调整均过大、说明超出歌手音域\n半音偏差：{mis_list}\n半音方差：{var_list}")
        merge.run(out_audio_name)
        # 清除缓存文件
        infer_tool.del_temp_wav("./wav_temp")

mis连续超过10%时，考虑升降半音

00429-01.wav: 33.33%  mis:0.94 var:1.15
00429-00.wav: 66.67%  mis:0.86 var:0.82
00429-02.wav: 100.0%  mis:0 var:0
分段误差参考：0.3优秀，0.5左右合理，少量0.8-1可以接受
若偏差过大，请调整升降半音数；多次调整均过大、说明超出歌手音域
半音偏差：[0.94, 0.86, 0]
半音方差：[1.15, 0.82, 0]
out sovits success


In [ ]:
import IPython.display as ipd
import torchaudio
#@markdown 预览干声（自行进results查看文件名，带.wav，仅支持wav）
source_path = "/content/sovits_f0_infer/results/vocals.wav"  #@param {type:"string"}
audio,sr = torchaudio.load(source_path)
ipd.display(ipd.Audio(audio, rate=sr))